In [ ]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

# .env 파일 불러오기
load_dotenv()

In [ ]:
BASE_NAME = "3-0f"  # <-- 이 값을 0-1, 0-5 등으로 변경
FRAME_TOLERANCE_FRAMES = 3.0 # <-- 이 값을 0.1, 0.5 등으로 변경
SYNC_LOG_PATH = os.path.join(os.getenv('BASE_PATH'), 'pbl_exp', 'data', f'sync_log_{BASE_NAME}_stable.csv')

FILE_TO_ANALYZE = f"sync_log_{BASE_NAME}_stable.csv"
TARGET_FPS = 15.0
FRAME_TOLERANCE_MS = (1.0 / TARGET_FPS) * 1000.0 * FRAME_TOLERANCE_FRAMES

# Set display options for clarity
pd.set_option('display.float_format', '{:.6f}'.format)

print(f"--- [Base Code] '{FILE_TO_ANALYZE}' 분석 시작 ---")
print(f"설정값: {FRAME_TOLERANCE_FRAMES} frame (15fps 기준, 최대 {FRAME_TOLERANCE_MS:.3f} ms 허용)")

try:
    if not os.path.exists(SYNC_LOG_PATH) or os.path.getsize(SYNC_LOG_PATH) == 0:
        raise FileNotFoundError(f"{FILE_TO_ANALYZE} 파일이 비어있거나 내용이 없습니다.")
        
    df = pd.read_csv(SYNC_LOG_PATH)
    
    if len(df) < 2:
        print("[오류] 분석을 위한 데이터가 부족합니다 (최소 2행 필요).")
    else:
        # --- 1. 유효 동기화 FPS (Effective Sync FPS) ---
        duration_sec = df['ts_zed'].max() - df['ts_zed'].min()
        num_events = len(df)
        effective_sync_fps = (num_events - 1) / duration_sec
        
        print("\n--- 1. 유효 동기화 FPS (처리량) ---")
        print(f"  안정화 구간 총 시간: {duration_sec:.4f} 초")
        print(f"  안정화 구간 이벤트 수: {num_events} 건")
        print(f"  [Key Metric] 유효 동기화 FPS: {effective_sync_fps:.4f} fps")
        print(f"  (목표 {TARGET_FPS} fps 대비 {effective_sync_fps / TARGET_FPS * 100:.2f} % 달성)")

        # --- 2. 동기화 오차 (Sync Error) ---
        # 'sys_diff_max(s)' 열의 통계를 ms 단위로 변환
        sync_error_ms = df['sys_diff_max(s)'] * 1000.0
        
        print("\n--- 2. 동기화 오차 (정확도 및 일관성) ---")
        print("  (단위: ms)")
        
        # .describe()로 Q1, Q3, min, max, std, mean 모두 계산
        stats = sync_error_ms.describe(percentiles=[.25, .5, .75])
        
        print(stats.to_string())
        
        print("\n  [Key Metrics 요약 (ms)]")
        print(f"  [Key Metric] Mean Sync Error    (평균): {stats['mean']:.3f} ms")
        print(f"  [Key Metric] Std.Dev Sync Error (표준편차): {stats['std']:.3f} ms")
        print(f"  [Key Metric] Max Sync Error     (최대): {stats['max']:.3f} ms")

        # --- 3. 이벤트 간격 (Jitter) ---
        # `ts_zed`의 차이를 계산 (ms 단위)
        event_jitter_ms = df['ts_zed'].diff() * 1000.0
        
        print("\n--- 3. 이벤트 간격 Jitter (안정성) ---")
        print("  (단위: ms)")
        
        jitter_stats = event_jitter_ms.dropna().describe(percentiles=[.25, .5, .75])
        print(jitter_stats.to_string())
        
        print(f"\n  (참고: {TARGET_FPS}fps의 목표 간격 = {1000.0 / TARGET_FPS:.3f} ms)")
        print(f"  [Key Metric] Jitter (Std.Dev): {jitter_stats['std']:.3f} ms")

except FileNotFoundError as e:
    print(f"[오류] {e}")
except Exception as e:
    print(f"분석 중 예외 발생: {e}")

print("\n--- [Base Code] 분석 완료 ---")

--- [Base Code] 'sync_log_3-0s_stable.csv' 분석 시작 ---
설정값: 3.0 frame (15fps 기준, 최대 200.000 ms 허용)

--- 1. 유효 동기화 FPS (처리량) ---
  안정화 구간 총 시간: 599.8739 초
  안정화 구간 이벤트 수: 8999 건
  [Key Metric] 유효 동기화 FPS: 14.9998 fps
  (목표 15.0 fps 대비 100.00 % 달성)

--- 2. 동기화 오차 (정확도 및 일관성) ---
  (단위: ms)
count   8999.000000
mean     170.354084
std       11.287865
min      141.035000
25%      162.486500
50%      171.727000
75%      179.608000
max      190.697000

  [Key Metrics 요약 (ms)]
  [Key Metric] Mean Sync Error    (평균): 170.354 ms
  [Key Metric] Std.Dev Sync Error (표준편차): 11.288 ms
  [Key Metric] Max Sync Error     (최대): 190.697 ms

--- 3. 이벤트 간격 Jitter (안정성) ---
  (단위: ms)
count   8998.000000
mean      66.667473
std        0.115712
min       64.261000
25%       66.616000
50%       66.633000
75%       66.746000
max       68.888000

  (참고: 15.0fps의 목표 간격 = 66.667 ms)
  [Key Metric] Jitter (Std.Dev): 0.116 ms

--- [Base Code] 분석 완료 ---


In [ ]:
# --- Configuration ---
FILE_TO_ANALYZE = f"tegra_log_1-0f_stable.csv"
TEGRA_LOG_PATH = os.path.join(os.getenv('BASE_PATH'), 'pbl_exp', 'data', FILE_TO_ANALYZE)
pd.set_option('display.float_format', '{:.3f}'.format)

print(f"--- [Base Code] '{FILE_TO_ANALYZE}' (시스템 부하) 분석 시작 ---")

try:
    if not os.path.exists(TEGRA_LOG_PATH) or os.path.getsize(TEGRA_LOG_PATH) == 0:
        raise FileNotFoundError(f"{FILE_TO_ANALYZE} 파일이 비어있거나 내용이 없습니다.")
        
    df = pd.read_csv(TEGRA_LOG_PATH)
    
    if len(df) == 0:
        print("[오류] 분석을 위한 데이터가 부족합니다.")
    else:
        print(f"  안정화 구간 총 {len(df)}초(행) 데이터 분석")

        # --- 1. CPU Load Analysis ---
        print("\n--- 1. CPU 사용률 (부하) ---")
        cpu_cols = [col for col in df.columns if col.startswith('CPU') and col.endswith('%')]
        
        if not cpu_cols:
            print("[오류] CPU 부하 열('CPU_..._%')을 찾을 수 없습니다.")
        else:
            # 1a. 전체 코어의 평균 (모든 코어, 모든 시간)
            all_cpu_data = df[cpu_cols].values.flatten()
            overall_cpu_mean = np.mean(all_cpu_data)
            
            print(f"  [Key Metric] 전체 CPU 평균 사용률: {overall_cpu_mean:.2f} %")

            # 1b. 코어별 평균 사용률 (멀티스레딩 확인)
            print("\n  [Insight] 코어별 평균 사용률 (부하 분산 확인):")
            core_means = df[cpu_cols].mean()
            print(core_means.to_string())

        # --- 2. GPU Load Analysis ---
        print("\n--- 2. GPU 사용률 ---")
        if 'GPU_Load_%' in df.columns:
            gpu_stats = df['GPU_Load_%'].describe(percentiles=[.25, .5, .75, .95])
            print(gpu_stats.to_string())
            print(f"  [Key Metric] 평균 GPU 사용률: {gpu_stats['mean']:.2f} %")
            if gpu_stats['50%'] == 0 and gpu_stats['95%'] > 0:
                print("  [Insight] GPU는 대부분(50% 이상) 유휴(0%) 상태이며, 필요시(95% percentile)에만 간헐적으로 사용됩니다.")
        else:
            print("[오류] 'GPU_Load_%' 열을 찾을 수 없습니다.")

        # --- 3. Memory Analysis ---
        print("\n--- 3. 메모리 사용량 ---")
        if 'RAM_Used_MB' in df.columns and 'SWAP_Used_MB' in df.columns:
            ram_stats = df['RAM_Used_MB'].describe()
            swap_stats = df['SWAP_Used_MB'].describe()
            
            print("  [RAM 사용량 (MB)]")
            print(ram_stats.to_string())
            print(f"  [Key Metric] 평균 RAM 사용량: {ram_stats['mean']:.1f} MB")

            print("\n  [SWAP 사용량 (MB)]")
            print(swap_stats.to_string())
            if swap_stats['max'] == 0:
                print("  [Insight] SWAP 사용량이 0으로, 10분간 메모리 누수(Leak)가 없었음을 확인했습니다.")
        else:
            print("[오류] 'RAM_Used_MB' 또는 'SWAP_Used_MB' 열을 찾을 수 없습니다.")

        # --- 4. Power Consumption Analysis ---
        print("\n--- 4. 시스템 전력 소모 (VIN) ---")
        if 'Power_VIN_mW' in df.columns:
            power_stats = df['Power_VIN_mW'].describe()
            print(power_stats.to_string())
            # mW를 W(와트)로 변환하여 출력
            print(f"  [Key Metric] 평균 시스템 전력: {power_stats['mean'] / 1000.0:.2f} W") 
        else:
            print("[오류] 'Power_VIN_mW' 열을 찾을 수 없습니다.")

        # --- 5. Temperature Analysis ---
        print("\n--- 5. 시스템 온도 ---")
        temp_col_to_use = 'Temp_TJ' if 'Temp_TJ' in df.columns else 'Temp_CPU'
        
        if temp_col_to_use in df.columns:
            print(f"  [주요 온도 지표: {temp_col_to_use} (C)]")
            temp_stats = df[temp_col_to_use].describe()
            print(temp_stats.to_string())
            print(f"  [Key Metric] 평균 온도: {temp_stats['mean']:.2f} C")
            print(f"  [Key Metric] 최대 온도: {temp_stats['max']:.2f} C")
            if temp_stats['max'] < 85.0:
                 print(f"  [Insight] 최대 온도가 {temp_stats['max']:.2f} C로 매우 안정적인 범위 내에서 관리되었습니다.")
        else:
            print("[오류] 'Temp_TJ' 또는 'Temp_CPU' 열을 찾을 수 없습니다.")

except FileNotFoundError as e:
    print(f"[오류] {e}")
except Exception as e:
    print(f"분석 중 예외 발생: {e}")

print("\n--- [Base Code] 분석 완료 ---")

--- [Base Code] 'tegra_log_1-0s_stable.csv' (시스템 부하) 분석 시작 ---
  안정화 구간 총 600초(행) 데이터 분석

--- 1. CPU 사용률 (부하) ---
  [Key Metric] 전체 CPU 평균 사용률: 10.46 %

  [Insight] 코어별 평균 사용률 (부하 분산 확인):
CPU1_%    12.902
CPU2_%     7.817
CPU3_%     7.840
CPU4_%     8.657
CPU5_%    12.383
CPU6_%    11.612
CPU7_%    10.428
CPU8_%    10.022
CPU9_%     9.433
CPU10_%   11.348
CPU11_%   10.522
CPU12_%   12.548

--- 2. GPU 사용률 ---
count   600.000
mean     13.032
std      18.807
min       0.000
25%       0.000
50%       0.000
75%      28.000
95%      49.000
max      52.000
  [Key Metric] 평균 GPU 사용률: 13.03 %
  [Insight] GPU는 대부분(50% 이상) 유휴(0%) 상태이며, 필요시(95% percentile)에만 간헐적으로 사용됩니다.

--- 3. 메모리 사용량 ---
  [RAM 사용량 (MB)]
count    600.000
mean    3577.708
std       14.014
min     3547.000
25%     3566.000
50%     3576.000
75%     3591.000
max     3602.000
  [Key Metric] 평균 RAM 사용량: 3577.7 MB

  [SWAP 사용량 (MB)]
count   600.000
mean      0.000
std       0.000
min       0.000
25%       0.000
50%       0.000
75%    